In [ ]:
from ratdata import data_manager as dm, process, ingest, plot as rdplot
from pathlib import Path
import numpy as np
import scipy.signal as signal

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
f_min = 2
f_max = 100
outfile = 'data/oof-%d-%d.txt' % (f_min, f_max)
with open(outfile, 'w') as f:
    f.write('filename,oof_exponent,stim\n')

all_recordings = dm.RecordingFile.select().count()
for i, r in enumerate(dm.RecordingFile.select()):
    if dm.is_recording_rejected(r.filename):
            continue
    if r.power.count() > 0:
        m = r.power.get().oof_exponent
    else:
        full_filename = Path(r.dirname) / r.filename
        data = ingest.read_mce_matlab_file(str(full_filename))
        if dm.is_recording_sliced(r.filename):
            slice = r.slice.get()
            slice_start = int(slice.start / data.dt)
            slice_end = slice_start + int(slice.length / data.dt)
            x = np.mean(data.electrode_data[:, slice_start:slice_end], 0)
        else:
            x = np.mean(data.electrode_data, 0)
        fs = int(1/data.dt)
        ff, pxx = signal.welch(x, fs, nperseg=fs)
        m, b = process.fit_oof(ff, pxx, f_min, f_max)
    if r.stim.count() != 0 and r.stim.get().stim_type != 'nostim':
        stim = r.stim.get().stim_type
    else:
        stim = 'nostim'
    print('[%d/%d] %s %.2f %s' % (i+1, all_recordings, r.filename, m, stim))
    with open(outfile, 'a') as f:
        f.write('%s,%.2f,%s\n' % (r.filename, m, stim))
        